In [ ]:
# Based on the excellent code by znxlwm (Hyeonwoo Kang)
# Modified with iPython plotting support & cleaned up by aluo-x (Andrew Luo)
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import os, time, itertools, imageio, pickle
from IPython import display

In [ ]:
def generator(z_in, condition_in, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        # For the input, we take in noise (z_in) + a digit (condition_in)
        cat_input = tf.concat([z_in, condition_in], 1)

        dense1 = tf.layers.dense(cat_input, units=128, activation=tf.nn.relu)

        dense2 = tf.layers.dense(dense1, units=128, activation=tf.nn.relu)

        dense3 = tf.layers.dense(dense2, units=784, activation=tf.nn.tanh)

        # We output after doing a bunch of dense transforms
        return dense3

In [ ]:
def discriminator(data_in, label_in, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # For the discriminator, it needs to take in data in (size 784) and digit (size 10)
        cat_input = tf.concat([data_in, label_in], 1)

        dense1 = tf.layers.dense(cat_input, units=128, activation=tf.nn.leaky_relu)

        dense2 = tf.layers.dense(dense1, units=128, activation=tf.nn.leaky_relu)

        dense3 = tf.layers.dense(dense2, units=1, activation=tf.nn.leaky_relu)

        return dense3

In [ ]:
onehot = np.eye(10)

def visualization_gen():
    # Generate data for visualization

    temp_z_ = np.random.normal(0, 1, (10, 100))
    fixed_z_ = temp_z_
    fixed_y_ = np.zeros((10, 1))

    for i in range(9):
        fixed_z_ = np.concatenate([fixed_z_, temp_z_], 0)
        temp = np.ones((10, 1)) + i
        fixed_y_ = np.concatenate([fixed_y_, temp], 0)
    fixed_y_ = onehot[fixed_y_.astype(np.int32)].squeeze()
    # Fixed z should be [10 by 100]
    # Fixed y should be [10 by 10]
    return fixed_z_, fixed_y_


fixed_z_, fixed_y_ = visualization_gen()

In [ ]:
batch_size = 100
lr = 0.0002
train_epoch = 100

In [ ]:
def show_result(num_epoch, show=False, save=False, path='result.png'):
    test_images = sess.run(G_z, {z: fixed_z_, y: fixed_y_})
    size_figure_grid = 10
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for k in range(size_figure_grid * size_figure_grid):
        i = k // size_figure_grid
        j = k % size_figure_grid
        ax[i, j].cla()
        ax[i, j].imshow(np.reshape(test_images[k], (28, 28)), cmap='gray')

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')
    if save:
        plt.savefig(path)
    if show:
        display.clear_output(wait=True)
        plt.show()
        display.display(plt.gcf())
        plt.gcf().clear()
    else:
        plt.close()


def show_train_hist(hist, show=False, save=False, path='Train_hist.png'):
    x = range(len(hist['D_losses']))
    y1 = hist['D_losses']
    y2 = hist['G_losses']
    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()
    if save:
        plt.savefig(path)
    if show:
        plt.show()
    else:
        plt.close()

In [ ]:
# load MNIST
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_set = (mnist.train.images - 0.5) / 0.5  # normalization; range: -1 ~ 1
train_label = mnist.train.labels

In [ ]:
# variables : input
x = tf.placeholder(tf.float32, shape=(None, 784))
y = tf.placeholder(tf.float32, shape=(None, 10))
z = tf.placeholder(tf.float32, shape=(None, 100))

In [ ]:
# networks : generator
G_z = generator(z, y)

# networks : discriminator
D_real_logits = discriminator(x, y)
D_fake_logits = discriminator(G_z, y, reuse=True)

In [ ]:
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones([batch_size, 1])))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros([batch_size, 1])))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones([batch_size, 1])))

In [ ]:
# trainable variables for each network
T_vars = tf.trainable_variables()
D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
G_vars = [var for var in T_vars if var.name.startswith('generator')]

In [ ]:
# optimizer for each network
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    D_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(D_loss, var_list=D_vars)
    G_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(G_loss, var_list=G_vars)

In [ ]:
# open session and initialize all variables
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [ ]:
# results save folder
root = 'MNIST_cGAN_results/'
model = 'MNIST_cGAN_'
if not os.path.isdir(root):
    os.mkdir(root)
if not os.path.isdir(root + 'Fixed_results'):
    os.mkdir(root + 'Fixed_results')

In [ ]:
train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []

In [ ]:
# training-loop
np.random.seed(int(time.time()))
print('training start!')
start_time = time.time()
for epoch in range(train_epoch):
    G_losses = []
    D_losses = []
    epoch_start_time = time.time()
    for iter in range(len(train_set) // batch_size):
        # update discriminator
        x_ = train_set[iter * batch_size:(iter + 1) * batch_size]
        y_ = train_label[iter * batch_size:(iter + 1) * batch_size]

        z_ = np.random.normal(0, 1, (batch_size, 100))

        loss_d_, _ = sess.run([D_loss, D_optim], {x: x_, y: y_, z: z_})
        D_losses.append(loss_d_)

        # update generator
        z_ = np.random.normal(0, 1, (batch_size, 100))
        y_ = np.random.randint(0, 9, (batch_size, 1))
        y_ = onehot[y_.astype(np.int32)].squeeze()
        loss_g_, _ = sess.run([G_loss, G_optim], {z: z_, x: x_, y: y_})
        G_losses.append(loss_g_)

    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time
    print('[%d/%d] - ptime: %.2f loss_d: %.3f, loss_g: %.3f' % (
    (epoch + 1), train_epoch, per_epoch_ptime, np.mean(D_losses), np.mean(G_losses)))
    fixed_p = root + 'Fixed_results/' + model + str(epoch + 1) + '.png'
    show_result((epoch + 1), show=True, save=True, path=fixed_p)
    train_hist['D_losses'].append(np.mean(D_losses))
    train_hist['G_losses'].append(np.mean(G_losses))
    train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

end_time = time.time()
total_ptime = end_time - start_time
train_hist['total_ptime'].append(total_ptime)

print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (
np.mean(train_hist['per_epoch_ptimes']), train_epoch, total_ptime))
print("Training finish!... save training results")
with open(root + model + 'train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

show_train_hist(train_hist, save=True, path=root + model + 'train_hist.png')

images = []
for e in range(train_epoch):
    img_name = root + 'Fixed_results/' + model + str(e + 1) + '.png'
    images.append(imageio.imread(img_name))
imageio.mimsave(root + model + 'generation_animation.gif', images, fps=5)

sess.close()